In [11]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
import torch
import numpy as np
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")
from seqm.seqm_functions.read_xyz import read_xyz
import scipy.io as sio
import math
import pandas as pd
import importlib
import matplotlib.pyplot as plt

import dftorch
importlib.reload(dftorch)
import dftorch.CoulombMatrix
importlib.reload(dftorch.CoulombMatrix)
from dftorch.CoulombMatrix import CoulombMatrix_vectorized
from dftorch.SCF import SCF, SCF_adaptive_mixing

from dftorch.H0andS import H0_and_S_vectorized
from dftorch.Constants import Constants
from dftorch.nearestneighborlist import vectorized_nearestneighborlist
from dftorch.Energy import Energy
from dftorch.Tools import fractional_matrix_power_symm
from dftorch.Forces import Forces
from dftorch.BondIntegral import *
from dftorch.Tools import ordered_pairs_from_TYPE

import matplotlib.colors as mcolors


torch.set_default_dtype(torch.float64)
print(torch.cuda.memory_allocated() / 1e9, 'GB')

0.0 GB


In [12]:
# Reload neighbor list module to pick up recent fixes
import importlib, dftorch.nearestneighborlist as _nnl
importlib.reload(_nnl)
from dftorch.nearestneighborlist import vectorized_nearestneighborlist
print("Reloaded dftorch.nearestneighborlist")

Reloaded dftorch.nearestneighborlist


In [13]:
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cpu'
const = Constants().to(device)

Efield = 0*0.3*torch.tensor([-.3,0.4,0.0], device=device).T # In arbitrary direction  Works ony in 0-field!!!
Te = 3000.0                       # Some electronic temperature in Kelvin, Possible bug at high tempertures!!!

species, coordinates = read_xyz(['COORD.xyz'], sort=False) #Input coordinate file
LBox = torch.tensor([8,4,4], device=device)


TYPE = torch.tensor(species[0], dtype=torch.int64, device=device)
RX = torch.tensor(coordinates[0,:,0], device=device).to(torch.get_default_dtype())
RY = torch.tensor(coordinates[0,:,1], device=device).to(torch.get_default_dtype())
RZ = torch.tensor(coordinates[0,:,2], device=device).to(torch.get_default_dtype())
Nocc = int(const.tore[TYPE].sum()/2)
Nats = len(RX)

# but first the neighborlist
#nrnnlist,nndist,nnRx,nnRy,nnRz,nnType,nnStruct,nrnnStruct  = vectorized_nearestneighborlist(RX,RY,RZ,LBox,4.0,Nats);
nrnnlist_no_self, nndist_no_self, nnRx_no_self, nnRy_no_self, nnRz_no_self, nnType_no_self, nnStruct_no_self, nrnnStruct_no_self  = \
        vectorized_nearestneighborlist(RX, RY, RZ, LBox, 6.0, Nats, upper_tri_only=True, remove_self_neigh=False);

# Reference gamma-point DFTB-SCC calculation
print("Running reference gamma-point calculation...")

# Get Hamiltonian, Overlap, atomic DM = D0 (vector only), etc
D0, H_INDEX_START, H_INDEX_END, Element_Type, Mnuc, Znuc, Hubbard_U, neighbor_I, neighbor_J, H0_SKF, dH0_SKF, S_SKF, dS_SKF = H0_and_S_vectorized(TYPE, RX, RY, RZ, LBox, Nats,
                                                                        nrnnlist_no_self, nnRx_no_self, nnRy_no_self, nnRz_no_self, nnType_no_self, const)

HDIM = max(H0_SKF.shape)

# Parameters for Coulomb summations
Rcut = 15.0
Coulomb_acc = 1e-10
TIMERATIO = 10

# Get full Coulomb matrix
nrnnlist, nndist, nnRx, nnRy, nnRz, nnType, nnStruct, nrnnStruct = vectorized_nearestneighborlist(RX, RY, RZ, LBox, Rcut, Nats,
                                                                                                   upper_tri_only=False, remove_self_neigh=False)
C, dCC = CoulombMatrix_vectorized(RX, RY, RZ, LBox, Hubbard_U, Element_Type, Nats, HDIM, Coulomb_acc, TIMERATIO, nnRx, nnRy, nnRz, nrnnlist, nnType,
                                  H_INDEX_START, H_INDEX_END)

# Run SCF calculation
H, Hcoul, Hdipole, D, q, f = SCF(H0_SKF.to(torch.float64), S_SKF.to(torch.float64), Efield.to(torch.float64), C.to(torch.float64), 
                            TYPE, RX.to(torch.float64), RY.to(torch.float64), RZ.to(torch.float64),
                            H_INDEX_START, H_INDEX_END,
                            Nocc, Hubbard_U, Znuc, Nats, Te, const, alpha=0.2, acc=1e-9, MAX_ITER=100)

# Calculate energy
Etot, Eband0, Ecoul, Edipole, S_ent = Energy(H0_SKF.to(torch.float64), Hubbard_U, Efield, D0, C.to(torch.float64), D, q,
                                         RX, RY, RZ, f, Te)

print(f"Incorrect results that the code currently gives:")
print(f"  Eband0 = {Eband0:.6f} eV")
print(f"  Total energy = {Etot:.6f} eV")
print(f"  Charge sum = {q.sum():.6f}")

Running reference gamma-point calculation...
H0_and_S
  Load H integral params
  Do H diagonal
  Do H off-diag
  Do H Slater-Koster SKF
  Do S Slater-Koster
  t 0.4 s

CoulombMatrix_vectorized
  Do Coulomb Real
  Coulomb_Real t 0.0 s

  Doing Coulomb k
   LMAX: 10
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
   10
  Coulomb_k t 0.1 s

### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
  update q 0.0 s
Res = 0.187740212, t = 0.0 s

Iter 2
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
  update q 0.0 s
Res = 0.143999880, t = 0.0 s

Iter 3
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
  update q 0.0 s
Res = 0.110455423, t = 0.0 s

Iter 4
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
  update q 0.0 s
Res = 0.084728637, t = 0.0 s

Iter 5
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
  update q 0.0 s
Res = 0.064996535, t = 0.0 s

Iter 6
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
  update q 0.0 s
Res = 0.

In [14]:
# Helpers: Monkhorst–Pack, SKF loading, SK blocks, k-space assembly
import torch, math
from typing import Dict, Tuple, List
from dftorch.Tools import ordered_pairs_from_TYPE
from dftorch.BondIntegral import read_skf_table, channels_to_matrix, cubic_spline_coeffs

# Use complex128 to avoid precision loss in Hermitian ops
cdtype = torch.complex128
fdtype = torch.float64

TWOPI = 2.0 * math.pi


def monkhorst_pack(n1: int, n2: int, n3: int, shift=(0,0,0), reduce_by_inversion=True):
    # fractional k in units of reciprocal lattice vectors, weights sum to 1
    g1 = torch.arange(n1, dtype=fdtype)
    g2 = torch.arange(n2, dtype=fdtype)
    g3 = torch.arange(n3, dtype=fdtype)
    s = torch.tensor(shift, dtype=fdtype)
    K = torch.stack(torch.meshgrid(g1, g2, g3, indexing='ij'), dim=-1).reshape(-1,3)
    K = (K + 0.5 + s) / torch.tensor([n1, n2, n3], dtype=fdtype)
    if reduce_by_inversion:
        # collapse pairs k and -k to Γ-centered set with doubled weights where needed
        modK = (K % 1.0)
        kept = []
        w = []
        used = torch.zeros(len(modK), dtype=torch.bool)
        for i in range(len(modK)):
            if used[i]:
                continue
            ki = modK[i]
            km = (-ki) % 1.0
            # find match
            jmatch = None
            for j in range(i+1, len(modK)):
                if not used[j] and torch.allclose(modK[j], km, atol=1e-12, rtol=0):
                    jmatch = j; break
            if jmatch is None:
                kept.append(ki)
                w.append(1.0)
                used[i] = True
            else:
                kept.append(ki)
                w.append(2.0)
                used[i] = True
                used[jmatch] = True
        kpts = torch.stack(kept, dim=0)
        wts = torch.tensor(w, dtype=fdtype)
    else:
        kpts = K
        wts = torch.ones(len(kpts), dtype=fdtype)
    wts = wts / wts.sum()
    return kpts.to(fdtype), wts.to(fdtype)


def build_skf_cache(TYPE, const) -> Dict[str, Tuple[torch.Tensor, torch.Tensor]]:
    _, _, labels = ordered_pairs_from_TYPE(TYPE, const)
    cache = {}
    for lab in labels:
        R, ch = read_skf_table(f"sk_orig/mio-1-1/mio-1-1/{lab}.skf")
        M = channels_to_matrix(ch)  # (n,20)
        coeffs = cubic_spline_coeffs(R, M)  # (n-1,20,4)
        cache[lab] = (R, coeffs)
    return cache


def eval_channels(cache, lab: str, r: torch.Tensor) -> torch.Tensor:
    # Evaluate all 20 channels at distance r (scalar). Returns (20,) values.
    R, coeffs = cache[lab]
    n = len(R)
    idx = torch.searchsorted(R, r, right=True) - 1
    idx = torch.clamp(idx, 0, n-2)
    dx = r - R[idx]
    a = coeffs[idx, :, 0]
    b = coeffs[idx, :, 1]
    c = coeffs[idx, :, 2]
    d = coeffs[idx, :, 3]
    y = a + b*dx + c*dx*dx + d*dx*dx*dx  # (20,)
    # hard cutoff: beyond last knot, set 0
    if r >= R[-1]:
        y = torch.zeros_like(y)
    return y


def assemble_H0S_kpoints(TYPE, H_INDEX_START, H_INDEX_END, H0_SKF, RX, RY, RZ, LBox,
                         nrnnlist, nnRx, nnRy, nnRz, nnType, nndist, Rcut_HS,
                         kpoints: torch.Tensor, skf_cache):
    # Returns lists of H0(k), S(k) for each kpoint
    HDIM = H0_SKF.shape[0]
    diag0 = torch.diagonal(H0_SKF).clone()
    # edges
    ei, ej, Tfrac, dR, Lc, Mc, Nc = build_edges_for_kspace(TYPE, RX, RY, RZ, LBox, nrnnlist, nnRx, nnRy, nnRz, nnType, nndist, Rcut_HS)
    # Precompute per-edge SK labels (both directions for heteronuclear handling)
    labels = const.label  # from kernel
    labs_ij = [f"{labels[int(TYPE[i])]}-{labels[int(TYPE[j])]}" for i,j in zip(ei.tolist(), ej.tolist())]
    labs_ji = [f"{labels[int(TYPE[j])]}-{labels[int(TYPE[i])]}" for i,j in zip(ei.tolist(), ej.tolist())]

    ai_st = H_INDEX_START.to(torch.int64)
    ai_en = H_INDEX_END.to(torch.int64)

    H0ks = []
    Sks = []
    for k in kpoints:
        Hk = torch.zeros((HDIM, HDIM), dtype=cdtype, device=RX.device)
        Sk = torch.eye(HDIM, dtype=cdtype, device=RX.device)
        Hk[range(HDIM), range(HDIM)] = diag0.to(cdtype)
        phase_arg = TWOPI * (Tfrac @ k)  # (nedges,)
        ph = torch.exp(1j * phase_arg.to(cdtype))
        for idx in range(len(ei)):
            i = int(ei[idx].item()); j = int(ej[idx].item())
            ist = int(ai_st[i].item()); ien = int(ai_en[i].item())
            jst = int(ai_st[j].item()); jen = int(ai_en[j].item())
            ni = ien - ist + 1; nj = jen - jst + 1
            Li, Mi, Ni = float(Lc[idx].item()), float(Mc[idx].item()), float(Nc[idx].item())
            # Channels for both directions
            y_ij = eval_channels(skf_cache, labs_ij[idx], dR[idx])
            y_ji = eval_channels(skf_cache, labs_ji[idx], dR[idx])
            # H channels
            Hss = y_ij[9];  Hsp_ij = y_ij[8];  Hpp_sig = y_ij[5];  Hpp_pi = y_ij[6]
            Hsp_ji = y_ji[8]
            # S channels (convert from eV to atomic-like scaling used in code)
            Sss = y_ij[19]/27.211396; Ssp_ij = y_ij[18]/27.211396; Spp_sig = y_ij[15]/27.211396; Spp_pi = y_ij[16]/27.211396
            Ssp_ji = y_ji[18]/27.211396
            # Place blocks
            if ni >= 1 and nj >= 1:
                Hk[ist+0, jst+0] += ph[idx] * Hss
                Sk[ist+0, jst+0] += ph[idx] * Sss
            if ni >= 1 and nj >= 4:
                Hk[ist+0, jst+1] += ph[idx] * (Li*Hsp_ij)
                Hk[ist+0, jst+2] += ph[idx] * (Mi*Hsp_ij)
                Hk[ist+0, jst+3] += ph[idx] * (Ni*Hsp_ij)
                Sk[ist+0, jst+1] += ph[idx] * (Li*Ssp_ij)
                Sk[ist+0, jst+2] += ph[idx] * (Mi*Ssp_ij)
                Sk[ist+0, jst+3] += ph[idx] * (Ni*Ssp_ij)
            if ni >= 4 and nj >= 1:
                Hk[ist+1, jst+0] += ph[idx] * (-Li*Hsp_ji)
                Hk[ist+2, jst+0] += ph[idx] * (-Mi*Hsp_ji)
                Hk[ist+3, jst+0] += ph[idx] * (-Ni*Hsp_ji)
                # Overlap s–p: reverse block should carry a minus sign (direction cosine flips)
                Sk[ist+1, jst+0] += ph[idx] * (-Li*Ssp_ji)
                Sk[ist+2, jst+0] += ph[idx] * (-Mi*Ssp_ji)
                Sk[ist+3, jst+0] += ph[idx] * (-Ni*Ssp_ji)
            if ni >= 4 and nj >= 4:
                PPSMPP = Hpp_sig - Hpp_pi
                SPPdiff = Spp_sig - Spp_pi
                # H pp block
                Hk[ist+1, jst+1] += ph[idx] * (Hpp_pi + Li*Li*PPSMPP)
                Hk[ist+1, jst+2] += ph[idx] * (Li*Mi*PPSMPP)
                Hk[ist+1, jst+3] += ph[idx] * (Li*Ni*PPSMPP)
                Hk[ist+2, jst+1] += ph[idx] * (Mi*Li*PPSMPP)
                Hk[ist+2, jst+2] += ph[idx] * (Hpp_pi + Mi*Mi*PPSMPP)
                Hk[ist+2, jst+3] += ph[idx] * (Mi*Ni*PPSMPP)
                Hk[ist+3, jst+1] += ph[idx] * (Ni*Li*PPSMPP)
                Hk[ist+3, jst+2] += ph[idx] * (Ni*Mi*PPSMPP)
                Hk[ist+3, jst+3] += ph[idx] * (Hpp_pi + Ni*Ni*PPSMPP)
                # S pp block
                Sk[ist+1, jst+1] += ph[idx] * (Spp_pi + Li*Li*SPPdiff)
                Sk[ist+1, jst+2] += ph[idx] * (Li*Mi*SPPdiff)
                Sk[ist+1, jst+3] += ph[idx] * (Li*Ni*SPPdiff)
                Sk[ist+2, jst+1] += ph[idx] * (Mi*Li*SPPdiff)
                Sk[ist+2, jst+2] += ph[idx] * (Spp_pi + Mi*Mi*SPPdiff)
                Sk[ist+2, jst+3] += ph[idx] * (Mi*Ni*SPPdiff)
                Sk[ist+3, jst+1] += ph[idx] * (Ni*Li*SPPdiff)
                Sk[ist+3, jst+2] += ph[idx] * (Ni*Mi*SPPdiff)
                Sk[ist+3, jst+3] += ph[idx] * (Spp_pi + Ni*Ni*SPPdiff)
        # enforce Hermitian by explicit sum (do not average)
        Hk = Hk + Hk.conj().T - torch.diag(torch.diagonal(Hk))
        Sk = Sk + Sk.conj().T - torch.diag(torch.diagonal(Sk))
        H0ks.append(Hk)
        Sks.append(Sk)
    return H0ks, Sks

In [32]:
# K-space edge builder and deduplicated assembler
import torch


def build_edges_for_kspace(TYPE, RX, RY, RZ, LBox, nrnnlist, nnRx, nnRy, nnRz, nnType, nndist, Rcut_HS, include_all_images=True):
    N = RX.shape[0]
    R = torch.stack([RX, RY, RZ], dim=1)

    edges = []  # list of tuples (i,j,Tfrac(ints), dR, L, M, N)

    if include_all_images:
        # Use actual image mapping for Tfrac, and evaluate integrals on the actual image vector (no rewrap)
        for i in range(N):
            m = int(nrnnlist[i].item())
            for t in range(m):
                j = int(nnType[i, t].item())
                if j < 0 or j < i:
                    continue
                pos = torch.tensor([nnRx[i, t], nnRy[i, t], nnRz[i, t]], dtype=RX.dtype)
                T = torch.round((pos - R[j]) / LBox).to(torch.int64)
                Tfrac = torch.tensor([int(T[0].item()), int(T[1].item()), int(T[2].item())], dtype=torch.int64)
                dvec = pos - R[i]
                dR = float(torch.norm(dvec).item())
                if dR <= 1e-12 or dR > Rcut_HS:
                    continue
                Lc = float((dvec[0] / dR).item())
                Mc = float((dvec[1] / dR).item())
                Nc = float((dvec[2] / dR).item())
                edges.append((i, j, Tfrac, dR, Lc, Mc, Nc))
    else:
        # deduplicate by choosing minimal-image per (i,j)
        best = {}
        tmp = []
        for i in range(N):
            m = int(nrnnlist[i].item())
            for t in range(m):
                j = int(nnType[i, t].item())
                if j < 0:
                    continue
                pos = torch.tensor([nnRx[i, t], nnRy[i, t], nnRz[i, t]], dtype=RX.dtype)
                dvec = pos - R[i]
                dR = float(torch.norm(dvec).item())
                if dR <= 1e-12 or dR > Rcut_HS:
                    continue
                tmp.append((i, j, t, dR))
                key = (min(i, j), max(i, j))
                if key not in best or dR < best[key][0]:
                    best[key] = (dR, len(tmp)-1)
        for (i0, j0), (dR, idx) in best.items():
            _i, _j, t, _ = tmp[idx]
            # ensure i<j ordering
            if _j < _i:
                _i, _j = _j, _i
            pos = torch.tensor([nnRx[_i, t], nnRy[_i, t], nnRz[_i, t]], dtype=RX.dtype)
            T = torch.round((pos - R[_j]) / LBox).to(torch.int64)
            Tfrac = torch.tensor([int(T[0].item()), int(T[1].item()), int(T[2].item())], dtype=torch.int64)
            dvec = pos - R[_i]
            Lc = float((dvec[0] / dR).item())
            Mc = float((dvec[1] / dR).item())
            Nc = float((dvec[2] / dR).item())
            edges.append((_i, _j, Tfrac, dR, Lc, Mc, Nc))

    if not edges:
        return (torch.empty(0, dtype=torch.int64), torch.empty(0, dtype=torch.int64),
                torch.empty((0,3), dtype=torch.int64), torch.empty(0),
                torch.empty(0), torch.empty(0), torch.empty(0))

    ei = torch.tensor([e[0] for e in edges], dtype=torch.int64)
    ej = torch.tensor([e[1] for e in edges], dtype=torch.int64)
    Tfrac = torch.stack([e[2] for e in edges], dim=0)
    dR = torch.tensor([e[3] for e in edges], dtype=RX.dtype)
    Lc = torch.tensor([e[4] for e in edges], dtype=RX.dtype)
    Mc = torch.tensor([e[5] for e in edges], dtype=RX.dtype)
    Nc = torch.tensor([e[6] for e in edges], dtype=RX.dtype)
    return ei, ej, Tfrac, dR, Lc, Mc, Nc


def assemble_H0S_kpoints_dedup(TYPE, H_INDEX_START, H_INDEX_END, H0_SKF, RX, RY, RZ, LBox,
                         nrnnlist, nnRx, nnRy, nnRz, nnType, nndist, Rcut_HS,
                         kpoints: torch.Tensor, skf_cache, include_all_images=True):
    HDIM = H0_SKF.shape[0]
    diag0 = torch.diagonal(H0_SKF).clone()
    ei, ej, Tfrac, dR, Lc, Mc, Nc = build_edges_for_kspace(TYPE, RX, RY, RZ, LBox, nrnnlist, nnRx, nnRy, nnRz, nnType, nndist, Rcut_HS, include_all_images=include_all_images)
    labels = const.label
    labs_ij = [f"{labels[int(TYPE[i])]}-{labels[int(TYPE[j])]}" for i,j in zip(ei.tolist(), ej.tolist())]
    labs_ji = [f"{labels[int(TYPE[j])]}-{labels[int(TYPE[i])]}" for i,j in zip(ei.tolist(), ej.tolist())]
    ai_st = H_INDEX_START.to(torch.int64)
    ai_en = H_INDEX_END.to(torch.int64)
    H0ks = []
    Sks = []
    for k in kpoints:
        Hk = torch.zeros((HDIM, HDIM), dtype=cdtype, device=RX.device)
        Sk = torch.eye(HDIM, dtype=cdtype, device=RX.device)
        Hk[range(HDIM), range(HDIM)] = diag0.to(cdtype)
        phase_arg = TWOPI * (Tfrac.to(fdtype) @ k)  # (nedges,)
        ph = torch.exp(1j * phase_arg.to(cdtype))
        for idx in range(len(ei)):
            i = int(ei[idx].item()); j = int(ej[idx].item())
            ist = int(ai_st[i].item()); ien = int(ai_en[i].item())
            jst = int(ai_st[j].item()); jen = int(ai_en[j].item())
            ni = ien - ist + 1; nj = jen - jst + 1
            Li, Mi, Ni = float(Lc[idx].item()), float(Mc[idx].item()), float(Nc[idx].item())
            y_ij = eval_channels(skf_cache, labs_ij[idx], dR[idx])
            y_ji = eval_channels(skf_cache, labs_ji[idx], dR[idx])
            Hss = y_ij[9];  Hsp_ij = y_ij[8];  Hpp_sig = y_ij[5];  Hpp_pi = y_ij[6]
            Hsp_ji = y_ji[8]
            Sss = y_ij[19]/27.211396; Ssp_ij = y_ij[18]/27.211396; Spp_sig = y_ij[15]/27.211396; Spp_pi = y_ij[16]/27.211396
            Ssp_ji = y_ji[18]/27.211396
            if ni >= 1 and nj >= 1:
                Hk[ist+0, jst+0] += ph[idx] * Hss
                Sk[ist+0, jst+0] += ph[idx] * Sss
            if ni >= 1 and nj >= 4:
                Hk[ist+0, jst+1] += ph[idx] * (Li*Hsp_ij)
                Hk[ist+0, jst+2] += ph[idx] * (Mi*Hsp_ij)
                Hk[ist+0, jst+3] += ph[idx] * (Ni*Hsp_ij)
                Sk[ist+0, jst+1] += ph[idx] * (Li*Ssp_ij)
                Sk[ist+0, jst+2] += ph[idx] * (Mi*Ssp_ij)
                Sk[ist+0, jst+3] += ph[idx] * (Ni*Ssp_ij)
            if ni >= 4 and nj >= 1:
                Hk[ist+1, jst+0] += ph[idx] * (-Li*Hsp_ji)
                Hk[ist+2, jst+0] += ph[idx] * (-Mi*Hsp_ji)
                Hk[ist+3, jst+0] += ph[idx] * (-Ni*Hsp_ji)
                # Overlap s–p: reverse block minus sign
                Sk[ist+1, jst+0] += ph[idx] * (-Li*Ssp_ji)
                Sk[ist+2, jst+0] += ph[idx] * (-Mi*Ssp_ji)
                Sk[ist+3, jst+0] += ph[idx] * (-Ni*Ssp_ji)
            if ni >= 4 and nj >= 4:
                PPSMPP = Hpp_sig - Hpp_pi
                SPPdiff = Spp_sig - Spp_pi
                Hk[ist+1, jst+1] += ph[idx] * (Hpp_pi + Li*Li*PPSMPP)
                Hk[ist+1, jst+2] += ph[idx] * (Li*Mi*PPSMPP)
                Hk[ist+1, jst+3] += ph[idx] * (Li*Ni*PPSMPP)
                Hk[ist+2, jst+1] += ph[idx] * (Mi*Li*PPSMPP)
                Hk[ist+2, jst+2] += ph[idx] * (Hpp_pi + Mi*Mi*PPSMPP)
                Hk[ist+2, jst+3] += ph[idx] * (Mi*Ni*PPSMPP)
                Hk[ist+3, jst+1] += ph[idx] * (Ni*Li*PPSMPP)
                Hk[ist+3, jst+2] += ph[idx] * (Ni*Mi*PPSMPP)
                Hk[ist+3, jst+3] += ph[idx] * (Hpp_pi + Ni*Ni*PPSMPP)
                Sk[ist+1, jst+1] += ph[idx] * (Spp_pi + Li*Li*SPPdiff)
                Sk[ist+1, jst+2] += ph[idx] * (Li*Mi*SPPdiff)
                Sk[ist+1, jst+3] += ph[idx] * (Li*Ni*SPPdiff)
                Sk[ist+2, jst+1] += ph[idx] * (Mi*Li*SPPdiff)
                Sk[ist+2, jst+2] += ph[idx] * (Spp_pi + Mi*Mi*SPPdiff)
                Sk[ist+2, jst+3] += ph[idx] * (Mi*Ni*SPPdiff)
                Sk[ist+3, jst+1] += ph[idx] * (Ni*Li*SPPdiff)
                Sk[ist+3, jst+2] += ph[idx] * (Ni*Mi*SPPdiff)
                Sk[ist+3, jst+3] += ph[idx] * (Spp_pi + Ni*Ni*SPPdiff)
        # Hermitize once (we only assembled i<j blocks)
        Hk = Hk + Hk.conj().T - torch.diag(torch.diagonal(Hk))
        Sk = Sk + Sk.conj().T - torch.diag(torch.diagonal(Sk))
        H0ks.append(Hk)
        Sks.append(Sk)
    return H0ks, Sks

In [25]:
# Reload DM_Fermi; ensure hermitian wrapper is available
import importlib
from dftorch import DM_Fermi as DMF
importlib.reload(DMF)
from dftorch.DM_Fermi import DM_Fermi_herm
print("Reloaded DM_Fermi; DM_Fermi_herm available, conjugate fix applied")

Reloaded DM_Fermi; DM_Fermi_herm available, conjugate fix applied


In [33]:
# Build k-meshes, assemble H0(k), S(k), and run SCF for Gamma and 3x3x3
Rcut_HS = 1e9  # include all neighbors from list
# k-meshes
kpoints_gamma = torch.zeros((1,3), dtype=fdtype)
kweights_gamma = torch.tensor([1.0], dtype=fdtype)
kpoints_333, kweights_333 = monkhorst_pack(3,3,3, shift=(0,0,0), reduce_by_inversion=False)

# SKF cache
skf_cache = build_skf_cache(TYPE, const)

# Toggle: reference-mode Γ assembly (minimal-image) vs full-image
GAMMA_REFERENCE_MODE = True

# Ensure DM_Fermi_herm is available
from dftorch.DM_Fermi import DM_Fermi_herm

if GAMMA_REFERENCE_MODE:
    H0ks_g, Sks_g = assemble_H0S_kpoints_dedup(TYPE, H_INDEX_START, H_INDEX_END, H0_SKF, RX, RY, RZ, LBox,
                         nrnnlist_no_self.squeeze(-1), nnRx_no_self, nnRy_no_self, nnRz_no_self, nnType_no_self, nndist_no_self, Rcut_HS,
                         kpoints_gamma, skf_cache, include_all_images=False)
    gamma_desc = "Gamma (reference-mode, minimal-image)"
else:
    H0ks_g, Sks_g = assemble_H0S_kpoints_dedup(TYPE, H_INDEX_START, H_INDEX_END, H0_SKF, RX, RY, RZ, LBox,
                         nrnnlist_no_self.squeeze(-1), nnRx_no_self, nnRy_no_self, nnRz_no_self, nnType_no_self, nndist_no_self, Rcut_HS,
                         kpoints_gamma, skf_cache, include_all_images=True)
    gamma_desc = "Gamma (full-image)"

# For k-mesh, also use minimal-image mapping to avoid overcounting duplicate images
H0ks_333, Sks_333 = assemble_H0S_kpoints_dedup(TYPE, H_INDEX_START, H_INDEX_END, H0_SKF, RX, RY, RZ, LBox,
                         nrnnlist_no_self.squeeze(-1), nnRx_no_self, nnRy_no_self, nnRz_no_self, nnType_no_self, nndist_no_self, Rcut_HS,
                         kpoints_333, skf_cache, include_all_images=False)

# SCF with plain linear mixing only

def SCF_kspace2(H0ks, Sks, kpoints, kweights, C, TYPE, RX, RY, RZ, H_INDEX_START, H_INDEX_END, Nocc, U, Znuc, Te,
                alpha=0.25, acc=5e-9, MAX_ITER=200):
    cdtype = torch.complex128
    fdtype = torch.float64
    n_orb_per_atom = (H_INDEX_END - H_INDEX_START + 1)
    atom_ids = torch.repeat_interleave(torch.arange(len(n_orb_per_atom), device=RX.device), n_orb_per_atom.to(torch.int64))
    nAO = atom_ids.shape[0]
    # Ortho per k
    Zks = []
    for Sk in Sks:
        Skh = 0.5 * (Sk + Sk.conj().T)
        evals, evecs = torch.linalg.eigh(Skh)
        evals = torch.clamp(evals.real, min=1e-12)
        Zk = evecs @ torch.diag((evals**(-0.5)).to(cdtype)) @ evecs.conj().T
        Zks.append(Zk)
    # Init density from H0 only
    Dks = []
    for H0k, Zk in zip(H0ks, Zks):
        Hh = 0.5 * (H0k + H0k.conj().T)
        Dorth, _ = DM_Fermi_herm(Zk.conj().T @ Hh @ Zk, Te, Nocc, mu_0=None, m=18, eps=1e-10, MaxIt=40)
        Dks.append(Zk @ Dorth @ Zk.conj().T)
    # Initial q from Mulliken: q = -Z + 2 * sum_k w_k diag(D S)
    q = -Znuc.clone().to(fdtype)
    DS_sum_AO = torch.zeros(nAO, dtype=fdtype, device=RX.device)
    for wk, Dk, Sk in zip(kweights, Dks, Sks):
        DS_k = 2.0 * (Dk @ Sk).diagonal().real
        DS_sum_AO += wk * DS_k
    q.scatter_add_(0, atom_ids, DS_sum_AO)

    it = 0
    Res = torch.tensor(float('inf'), dtype=fdtype)
    while Res > acc and it < MAX_ITER:
        it += 1
        CoulPot = (C.to(fdtype)) @ q
        V_A = (U.to(fdtype)) * q + CoulPot
        V_ao = V_A[atom_ids]
        Hks = []
        for H0k, Sk in zip(H0ks, Sks):
            Hcoul = 0.5 * (V_ao.unsqueeze(1) * Sk + Sk * V_ao.unsqueeze(0))
            Hks.append(H0k + Hcoul)
        Dks_new = []
        q_out = -Znuc.clone().to(fdtype)
        DS_sum_AO = torch.zeros(nAO, dtype=fdtype, device=RX.device)
        for wk, Hk, Sk, Zk in zip(kweights, Hks, Sks, Zks):
            Hh = 0.5 * (Hk + Hk.conj().T)
            Dorth, _ = DM_Fermi_herm(Zk.conj().T @ Hh @ Zk, Te, Nocc, mu_0=None, m=18, eps=1e-10, MaxIt=40)
            Dk = Zk @ Dorth @ Zk.conj().T
            Dks_new.append(Dk)
            DS_k = 2.0 * (Dk @ Sk).diagonal().real
            DS_sum_AO += wk * DS_k
        q_out.scatter_add_(0, atom_ids, DS_sum_AO)
        Res = torch.norm(q_out - q)
        print(f"Iter {it:3d}: Res={Res.item():.3e}")
        q = (1.0 - alpha) * q + alpha * q_out
        Dks = Dks_new
    # Energies (report 2 Σ_k w Tr[H0(k) D(k)] as Eband0)
    Eband0 = 0.0
    for wk, H0k, Dk in zip(kweights, H0ks, Dks):
        Eband0 += wk * (2.0 * torch.trace((0.5*(H0k+H0k.conj().T)) @ Dk)).real.item()
    Ecoul = 0.5 * (q @ (C.to(fdtype) @ q)).item() + 0.5 * torch.sum((U.to(fdtype) * q) * q).item()
    return {'Eband0': Eband0, 'Ecoul': Ecoul, 'q': q, 'iters': it, 'Res': Res.item(), 'Dks': Dks}

# Run SCF and print energies
print(f"Running SCF for {gamma_desc}...")
res_gamma = SCF_kspace2(H0ks_g, Sks_g, kpoints_gamma, kweights_gamma, C, TYPE, RX, RY, RZ, H_INDEX_START, H_INDEX_END, Nocc, Hubbard_U, Znuc, Te)
print(f"{gamma_desc}: Eband0={res_gamma['Eband0']:.6f} eV, Ecoul={res_gamma['Ecoul']:.6f} eV, iters={res_gamma['iters']}, Res={res_gamma['Res']:.2e}")

print("Running SCF for 3x3x3 k-mesh...")
res_333 = SCF_kspace2(H0ks_333, Sks_333, kpoints_333, kweights_333, C, TYPE, RX, RY, RZ, H_INDEX_START, H_INDEX_END, Nocc, Hubbard_U, Znuc, Te)
print(f"3x3x3: Eband0={res_333['Eband0']:.6f} eV, Ecoul={res_333['Ecoul']:.6f} eV, iters={res_333['iters']}, Res={res_333['Res']:.2e}")

# Dks for reporting
Dks_g = res_gamma['Dks']
Dks_333 = res_333['Dks']

Running SCF for Gamma (reference-mode, minimal-image)...
Iter   1: Res=1.877e-01
Iter   2: Res=1.331e-01
Iter   3: Res=9.432e-02
Iter   4: Res=6.686e-02
Iter   5: Res=4.740e-02
Iter   6: Res=3.361e-02
Iter   7: Res=2.383e-02
Iter   8: Res=1.690e-02
Iter   9: Res=1.198e-02
Iter  10: Res=8.496e-03
Iter  11: Res=6.025e-03
Iter  12: Res=4.273e-03
Iter  13: Res=3.031e-03
Iter  14: Res=2.149e-03
Iter  15: Res=1.525e-03
Iter  16: Res=1.082e-03
Iter  17: Res=7.673e-04
Iter  18: Res=5.443e-04
Iter  19: Res=3.862e-04
Iter  20: Res=2.740e-04
Iter  21: Res=1.945e-04
Iter  22: Res=1.380e-04
Iter  23: Res=9.794e-05
Iter  24: Res=6.952e-05
Iter  25: Res=4.935e-05
Iter  26: Res=3.503e-05
Iter  27: Res=2.487e-05
Iter  28: Res=1.766e-05
Iter  29: Res=1.254e-05
Iter  30: Res=8.907e-06
Iter  31: Res=6.327e-06
Iter  32: Res=4.494e-06
Iter  33: Res=3.193e-06
Iter  34: Res=2.269e-06
Iter  35: Res=1.612e-06
Iter  36: Res=1.146e-06
Iter  37: Res=8.146e-07
Iter  38: Res=5.792e-07
Iter  39: Res=4.118e-07
Iter  4

In [18]:
# Energy helpers (required by final report)
fd = torch.float64
cd = torch.complex128

# Build AO-diagonal potential from per-atom q directly using H_INDEX_START/END

def build_Vao_from_q(q, Hubbard_U, C, H_INDEX_START, H_INDEX_END):
    Vatom = (Hubbard_U.view(-1).to(fd) * q.view(-1).to(fd)) + C.to(fd) @ q.view(-1).to(fd)
    dim = int(H_INDEX_END[-1].item() + 1)
    Vao = torch.zeros((dim, dim), dtype=fd)
    for i in range(len(H_INDEX_START)):
        ist = int(H_INDEX_START[i].item()); ien = int(H_INDEX_END[i].item())
        Vao[ist:ien+1, ist:ien+1] = torch.eye(ien - ist + 1, dtype=fd) * Vatom[i]
    return Vao


def to_cd(x):
    return x.to(cd) if not torch.is_complex(x) else x


def eigh_gen(H, S):
    S = 0.5*(to_cd(S) + to_cd(S).mH)
    H = 0.5*(to_cd(H) + to_cd(H).mH)
    evals, U = torch.linalg.eigh(S)
    Sinvh = U @ torch.diag(evals.real.clamp_min(1e-18).rsqrt()).to(cd) @ U.mH  # S^{-1/2}
    Sh   = U @ torch.diag(evals.real.clamp_min(1e-18).sqrt()).to(cd) @ U.mH     # S^{+1/2}
    Htil = Sinvh @ H @ Sinvh
    e, V = torch.linalg.eigh(Htil)
    return e.real.to(fd), V.to(cd), Sinvh, Sh


def occ_from_D_in_eigbasis(D, S, V, Sinvh, Sh):
    # Project AO density to orthonormal basis: Dorth = S^{-1/2} D S^{-1/2}
    Dorth = Sinvh @ to_cd(D) @ Sinvh
    return torch.diagonal(V.mH @ Dorth @ V).real.to(fd)


def eband_total_from_eigs(Hks, Sks, Dks, wk, Vao):
    def Htot_k(Hk, Sk):
        Skc = to_cd(Sk)
        Vc = Vao.to(cd)
        Hcoulk = 0.5*(Vc @ Skc + Skc @ Vc)
        return (Hk.to(cd) if not torch.is_complex(Hk) else Hk) + Hcoulk
    Etot = 0.0
    for Hk,Sk,Dk,w in zip(Hks,Sks,Dks,wk):
        Ht = Htot_k(Hk, Sk)
        e, V, Sph, Sh = eigh_gen(Ht, Sk)
        fill = occ_from_D_in_eigbasis(Dk, Sk, V, Sph, Sh).clamp(min=0.0, max=2.0)
        Etot += float((w * (e * fill).sum()).item())
    return 2.0 * Etot

In [36]:
# Compact energy summary using eig-based band sum helper
# E_band_total = 2 * sum_k w_k sum_i f_i(k) * eps_i(k) with H = H0 + Hcoul
# E_H0D       = 2 * sum_k w_k Tr[H0(k) D(k)]
# E_HcD       = E_band_total - E_H0D
# EnonSCC     = E_H0D - 2 Tr[H0 D0]
# E_total(DFTB conv.) = EnonSCC + Ecoul  [entropy omitted here]

# Gamma
E_band_tot_g = eband_total_from_eigs(H0ks_g, Sks_g, res_gamma['Dks'], kweights_gamma, Vao)
E_H0D_g = 0.0
for wk, H0k, Dk in zip(kweights_gamma, H0ks_g, res_gamma['Dks']):
    H0h = 0.5 * (H0k + H0k.conj().T)
    E_H0D_g += wk * (2.0 * torch.trace(H0h @ Dk)).real.item()
E_HcD_g = E_band_tot_g - E_H0D_g
EnonSCC_g = E_H0D_g - E_H0D0
Etot_conv_g = EnonSCC_g + res_gamma['Ecoul']

# 3x3x3
E_band_tot_333 = eband_total_from_eigs(H0ks_333, Sks_333, res_333['Dks'], kweights_333, Vao333)
E_H0D_333 = 0.0
for wk, H0k, Dk in zip(kweights_333, H0ks_333, res_333['Dks']):
    H0h = 0.5 * (H0k + H0k.conj().T)
    E_H0D_333 += wk * (2.0 * torch.trace(H0h @ Dk)).real.item()
E_HcD_333 = E_band_tot_333 - E_H0D_333
EnonSCC_333 = E_H0D_333 - E_H0D0
Etot_conv_333 = EnonSCC_333 + res_333['Ecoul']

print("Gamma totals:")
print(f"  2*sum_k w sum_i f_i eps_i = {E_band_tot_g: .6f} eV")
print(f"  2*sum_k w Tr[H0 D]        = {E_H0D_g: .6f} eV")
print(f"  2*Tr[H0 D0] (reference)   = {E_H0D0: .6f} eV")
print(f"  EnonSCC = 2Tr[H0(D-D0)]   = {EnonSCC_g: .6f} eV")
print(f"  2*sum_k w Tr[Hcoul D]     = {E_HcD_g: .6f} eV")
print(f"  Ecoul (1/2 q^T γ q)       = {res_gamma['Ecoul']: .6f} eV")
print(f"  E_total (DFTB conv.)      = {Etot_conv_g: .6f} eV")

print("3x3x3 totals:")
print(f"  2*sum_k w sum_i f_i eps_i = {E_band_tot_333: .6f} eV")
print(f"  2*sum_k w Tr[H0 D]        = {E_H0D_333: .6f} eV")
print(f"  2*Tr[H0 D0] (reference)   = {E_H0D0: .6f} eV")
print(f"  EnonSCC = 2Tr[H0(D-D0)]   = {EnonSCC_333: .6f} eV")
print(f"  2*sum_k w Tr[Hcoul D]     = {E_HcD_333: .6f} eV")
print(f"  Ecoul (1/2 q^T γ q)       = {res_333['Ecoul']: .6f} eV")
print(f"  E_total (DFTB conv.)      = {Etot_conv_333: .6f} eV")


Gamma totals:
  2*sum_k w sum_i f_i eps_i = -115.160220 eV
  2*sum_k w Tr[H0 D]        = -325.822967 eV
  2*Tr[H0 D0] (reference)   = -256.102287 eV
  EnonSCC = 2Tr[H0(D-D0)]   = -69.720680 eV
  2*sum_k w Tr[Hcoul D]     =  210.662747 eV
  Ecoul (1/2 q^T γ q)       =  0.243208 eV
  E_total (DFTB conv.)      = -69.477472 eV
3x3x3 totals:
  2*sum_k w sum_i f_i eps_i = -115.166519 eV
  2*sum_k w Tr[H0 D]        = -325.943791 eV
  2*Tr[H0 D0] (reference)   = -256.102287 eV
  EnonSCC = 2Tr[H0(D-D0)]   = -69.841504 eV
  2*sum_k w Tr[Hcoul D]     =  210.777272 eV
  Ecoul (1/2 q^T γ q)       =  0.240617 eV
  E_total (DFTB conv.)      = -69.600886 eV


In [20]:
# Compact energy summary matching E = 2 Tr[H0 (D - D0)] + Ecoul - 2 Te S
kB = 8.61739e-5

def entropy_from_Dks(Dks, Sks, kweights):
    S = 0.0
    for wk, Dk, Sk in zip(kweights, Dks, Sks):
        # orthonormalize to get Dorth and occupations
        evals, evecs = torch.linalg.eigh(0.5*(Sk+Sk.conj().T))
        Sinvh = evecs @ torch.diag(evals.real.clamp_min(1e-18).rsqrt()).to(torch.complex128) @ evecs.conj().T
        Dorth = Sinvh @ (Dk.to(torch.complex128)) @ Sinvh
        f = torch.linalg.eigvalsh(0.5*(Dorth + Dorth.conj().T)).real.clamp(min=1e-14, max=1-1e-14)
        S += float(wk * (-torch.sum(f*torch.log(f) + (1-f)*torch.log(1-f))).item())
    return S

# Band energies from k-space H0 and D, then subtract 2 Tr[H0 D0]
Eband_g = 0.0
for wk, H0k, Dk in zip(kweights_gamma, H0ks_g, Dks_g):
    Eband_g += wk * (2.0 * torch.trace((0.5*(H0k+H0k.conj().T)) @ Dk)).real.item()
Eband_333 = 0.0
for wk, H0k, Dk in zip(kweights_333, H0ks_333, Dks_333):
    Eband_333 += wk * (2.0 * torch.trace((0.5*(H0k+H0k.conj().T)) @ Dk)).real.item()

# Reference subtraction term using real-space H0 and atomic D0 (vector)
if D0.ndim == 2:
    D0_diag = torch.diag(D0)
else:
    D0_diag = D0
Eref = 2.0 * float(torch.sum(torch.diagonal(H0_SKF) * D0_diag).item())

# Entropy contributions from occupations in orthonormal basis
S_g = entropy_from_Dks(Dks_g, Sks_g, kweights_gamma)
S_333 = entropy_from_Dks(Dks_333, Sks_333, kweights_333)

# Coulomb energies from converged q
Ecoul_g = 0.5 * (res_gamma['q'] @ (C.to(torch.float64) @ res_gamma['q'])).item() + 0.5 * torch.sum((Hubbard_U.to(torch.float64) * res_gamma['q']) * res_gamma['q']).item()
Ecoul_333 = 0.5 * (res_333['q'] @ (C.to(torch.float64) @ res_333['q'])).item() + 0.5 * torch.sum((Hubbard_U.to(torch.float64) * res_333['q']) * res_333['q']).item()

Etot_g = (Eband_g - Eref) + Ecoul_g - 2.0 * Te * S_g
Etot_333 = (Eband_333 - Eref) + Ecoul_333 - 2.0 * Te * S_333

print(f"Energy summary (with 2Tr[H0(D-D0)] + Ecoul - 2Te S):")
print(f"  {gamma_desc}: Eband0={Eband_g - Eref:.6f} eV, Ecoul={Ecoul_g:.6f} eV, S={S_g:.6f}, Etot={Etot_g:.6f} eV")
print(f"  3x3x3: Eband0={Eband_333 - Eref:.6f} eV, Ecoul={Ecoul_333:.6f} eV, S={S_333:.6f}, Etot={Etot_333:.6f} eV")

Energy summary (with 2Tr[H0(D-D0)] + Ecoul - 2Te S):
  Gamma (reference-mode, minimal-image): Eband0=-69.720680 eV, Ecoul=0.243208 eV, S=8.025824, Etot=-48224.419357 eV
  3x3x3: Eband0=7862729258326.827148 eV, Ecoul=1920082697681728438272.000000 eV, S=0.000000, Etot=1920082705544457748480.000000 eV


In [21]:
# Debug k-grid weights and assembly stats
print("kpoints_333 shape:", kpoints_333.shape, "dtype:", kpoints_333.dtype)
print("kweights_333 shape:", kweights_333.shape, "dtype:", kweights_333.dtype)
print("kweights_333 sum:", float(kweights_333.sum().item()))
print("kweights_333 min/max:", float(kweights_333.min().item()), float(kweights_333.max().item()))
print("First 5 kweights:", kweights_333[:5].tolist())
# Sanity on H0ks/Sks sizes
print("len(H0ks_333)=", len(H0ks_333), "len(Sks_333)=", len(Sks_333))
# Check a sample S eigenvalues to ensure positive definiteness
import numpy as np
if len(Sks_333) > 0:
    Sk0 = Sks_333[0].cpu()
    wS, _ = torch.linalg.eigh(Sk0.real)  # Sk is Hermitian
    print("Sk0 eigs min/max:", float(wS.min().item()), float(wS.max().item()))


kpoints_333 shape: torch.Size([27, 3]) dtype: torch.float64
kweights_333 shape: torch.Size([27]) dtype: torch.float64
kweights_333 sum: 1.0
kweights_333 min/max: 0.037037037037037035 0.037037037037037035
First 5 kweights: [0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035]
len(H0ks_333)= 27 len(Sks_333)= 27
Sk0 eigs min/max: 0.18043670836118372 2.027403470595937


In [22]:
# Inspect SCF_kspace2 source to verify energy accumulation
import inspect
print(inspect.getsource(SCF_kspace2))

def SCF_kspace2(H0ks, Sks, kpoints, kweights, C, TYPE, RX, RY, RZ, H_INDEX_START, H_INDEX_END, Nocc, U, Znuc, Te,
                alpha=0.25, acc=5e-9, MAX_ITER=200):
    cdtype = torch.complex128
    fdtype = torch.float64
    n_orb_per_atom = (H_INDEX_END - H_INDEX_START + 1)
    atom_ids = torch.repeat_interleave(torch.arange(len(n_orb_per_atom), device=RX.device), n_orb_per_atom.to(torch.int64))
    nAO = atom_ids.shape[0]
    # Ortho per k
    Zks = []
    for Sk in Sks:
        Skh = 0.5 * (Sk + Sk.conj().T)
        evals, evecs = torch.linalg.eigh(Skh)
        evals = torch.clamp(evals.real, min=1e-12)
        Zk = evecs @ torch.diag((evals**(-0.5)).to(cdtype)) @ evecs.conj().T
        Zks.append(Zk)
    # Init density from H0 only
    Dks = []
    for H0k, Zk in zip(H0ks, Zks):
        Hh = 0.5 * (H0k + H0k.conj().T)
        Dorth, _ = DM_Fermi_herm(Zk.conj().T @ Hh @ Zk, Te, Nocc, mu_0=None, m=18, eps=1e-10, MaxIt=40)
        Dks.append(Zk @ Dorth @ Zk.conj().T)
    # Initial 

In [28]:
# Diagnose 3x3x3 explosion: norms and ranges
import math
print("Electrons check:")
for name, res in [("gamma", res_gamma), ("333", res_333)]:
    Dks = res['Dks']
    Sks = Sks_g if name=="gamma" else Sks_333
    kwe = kweights_gamma if name=="gamma" else kweights_333
    nelec = 0.0
    for wk, Dk, Sk in zip(kwe, Dks, Sks):
        nelec += wk.item() * (2.0 * torch.trace(Dk @ Sk).real.item())
    print(name, "nelec=", nelec)

# Max/min of H0 and D for 333
max_H0 = 0.0
max_D = 0.0
for H0k in H0ks_333:
    max_H0 = max(max_H0, float(torch.max(torch.abs(H0k)).item()))
for Dk in res_333['Dks']:
    max_D = max(max_D, float(torch.max(torch.abs(Dk)).item()))
print("max |H0k| for 333:", max_H0)
print("max |Dk| for 333:", max_D)

# Check a few phase factors magnitudes (should be 1)
if len(H0ks_333)>0:
    ei, ej, Tfrac, dR, Lc, Mc, Nc = build_edges_for_kspace(TYPE, RX, RY, RZ, LBox, nrnnlist, nnRx, nnRy, nnRz, nnType, nndist, Rcut_HS, include_all_images=True)
    k = kpoints_333[0]
    phase_arg = TWOPI * (Tfrac.to(fdtype) @ k)
    ph = torch.exp(1j * phase_arg.to(cdtype))
    mag = torch.max(torch.abs(ph)).item()
    print("phase |ph| max:", mag)


Electrons check:
gamma nelec= 32.0
333 nelec= -425447373434.02
max |H0k| for 333: 13.73881005
max |Dk| for 333: 167301264177.05298
phase |ph| max: 1.0


In [24]:
# Scan S eigenvalues across 3x3x3
mins = []
maxs = []
for Sk in Sks_333:
    Skh = 0.5*(Sk+Sk.conj().T)
    wS, _ = torch.linalg.eigh(Skh.real)
    mins.append(float(wS.min().item()))
    maxs.append(float(wS.max().item()))
print("Sks_333 eigen min over k:", min(mins), "max over k:", max(maxs))


Sks_333 eigen min over k: -0.2970442797553265 max over k: 2.46435320884616
